In [ ]:
import pandas as pd

BEHAVIOR_COLUMNS = ['ImpressionID', 'UserID', 'Time', 'History', 'Impressions']

# 헤더 없음 → header=None 이 안전
train = pd.read_csv('download/MINDsmall_train/behaviors.tsv', sep='\t',
                    names=BEHAVIOR_COLUMNS, header=None)
dev   = pd.read_csv('download/MINDsmall_dev/behaviors.tsv', sep='\t',
                    names=BEHAVIOR_COLUMNS, header=None)

behaviors_df = pd.concat([train, dev], ignore_index=True)

# 1) 공백 분리 후 explode
ex = (behaviors_df[['UserID', 'Impressions']]
      .assign(Impressions=lambda d: d['Impressions'].fillna('').astype(str))
      .assign(token=lambda d: d['Impressions'].str.split())
      .explode('token')
      .query("token != ''"))

# 2) "newsId-label" 안전 파싱 (뒤에서 1번만 split)
parsed = ex['token'].str.rsplit('-', n=1, expand=True)
ex = ex.assign(NewsID=parsed[0].str.strip(),
               Label=parsed[1].str.strip()).drop(columns=['token', 'Impressions'])

# 3) 클릭만 (-1) 필터
clicked = ex.query("Label == '1'")

# 전체에서 '클릭된 뉴스ID'만, 중복 제거
clicked_news_unique = clicked['NewsID'].drop_duplicates().reset_index(drop=True)

print("고유 클릭 뉴스 개수:", len(clicked_news_unique))
print(clicked_news_unique.head())


고유 클릭 뉴스 개수: 9100
0    N55689
1    N17059
2    N23814
3    N49685
4     N8400
Name: NewsID, dtype: object

유저-뉴스 클릭 고유 쌍 개수: 345188
   UserID  NewsID
0  U13740  N55689
1  U91836  N17059
2  U73700  N23814
3  U34670  N49685
4   U8125   N8400
